In [13]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
import os
from scipy.stats import pearsonr

In [9]:
def format_val_file(file_val, index_col):
    """
    The columns is kid of different from the 'shuffled_20220317.csv'
    :param file_val: File for validate.
    :return: X_val and y_val
    """
    df_val = pd.read_csv(file_val, index_col=index_col)
    dataset2 = pd.to_datetime(df_val['Date'])
    DOY = dataset2.dt.dayofyear
    Year = dataset2.dt.year
    df_val.insert(0, 'DOY', DOY)
    df_val.insert(0, 'Year', Year)
    df_val.drop(labels=['Date', 'station', 'ESA-CCI', 'network'], axis=1, inplace=True)
    y_val = df_val.iloc[:, -2].values
    df_val.drop(labels='Soil Moisture', axis=1, inplace=True)
    X_val = df_val.iloc[:, :].values

    return X_val, y_val

In [10]:
file = 'ML_training&testing_v01shuffled_20220317.csv'
file_val = 'ML_validating_v01_20220303.csv'
folder_independent = 'output'
df = pd.read_csv(file)
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values
# Split the training&testing data into training dataset and testing dataset, respectively.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

In [11]:
reg_AB = AdaBoostRegressor(DecisionTreeRegressor(criterion='squared_error',max_depth=10),n_estimators=30,learning_rate=0.2)
ereg = reg_AB.fit(X_train,y_train)

In [14]:
%%time
pred = ereg.predict(X_test)
result = pd.DataFrame([y_test,pred],index=["y_test","y_pred"])
result.T.to_csv('test_random.csv')
pearson_r = pearsonr(y_test, pred)[0]
mse = mean_squared_error(y_test, pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, pred)
print('Testing: ')
print(f'MSE: {mse}, RMSE: {rmse}, r2: {r2}, Pearson_r:, {pearson_r}')
print('--------------- \n\n')

Testing: 
MSE: 0.004232425326445237, RMSE: 0.0650570928219609, r2: 0.6597022843808604, Pearson_r:, 0.8222471159679575
--------------- 


CPU times: user 6.37 s, sys: 133 ms, total: 6.5 s
Wall time: 6.63 s


In [15]:
%%time
X_val, y_val = format_val_file(file_val, index_col=None)
pred_val = ereg.predict(X_val)
result = pd.DataFrame([y_val, pred_val],index=["y_test","y_pred"])
result.T.to_csv('test_temporal.csv')
mse = mean_squared_error(y_val, pred_val)
rmse = np.sqrt(mse)
r2 = r2_score(y_val, pred_val)
pearson_r = pearsonr(y_val, pred_val)[0]
print('Validation: ')
print(f'MSE: {mse}, RMSE: {rmse}, r2: {r2}, Pearson_r:, {pearson_r}')
print('--------------- \n\n')

Validation: 
MSE: 0.004844571625660459, RMSE: 0.06960295701807832, r2: 0.6113744998867858, Pearson_r:, 0.7884322060696809
--------------- 


CPU times: user 11.5 s, sys: 318 ms, total: 11.8 s
Wall time: 12.4 s


In [18]:
%%time
files = os.listdir(folder_independent)
df_independent_metrics = pd.DataFrame(columns=['network', 'station', 'MSE', 'RMSE', 'R2', 'Pearson_r', 'n_size'],
                                          dtype='object')
for idx, file in enumerate(files):
    # Read the data from independent stations.
    X_val_in, y_val_in = format_val_file(os.path.join(folder_independent, file), index_col=0)
    pred_val_in = ereg.predict(X_val_in)
    result = pd.DataFrame([y_val_in,pred_val_in],index=["y_test","y_pred"])
    result.T.to_csv(file[:-4] + '_ada.csv')
    mse = mean_squared_error(y_val_in, pred_val_in)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_val_in, pred_val_in)
    pearson_r = pearsonr(y_val_in, pred_val_in)[0]

    df_val_in = pd.read_csv(os.path.join(folder_independent, file))

    s_val_in = pd.Series(index=['network', 'station', 'MSE', 'RMSE', 'R2', 'Pearson_r', 'n_size'], dtype='object')
    s_val_in['network'] = file.split('_')[1]
    s_val_in['station'] = file.split('_')[2]
    s_val_in['MSE'] = mse
    s_val_in['RMSE'] = rmse
    s_val_in['R2'] = r2
    s_val_in['Pearson_r'] = pearson_r
    s_val_in['n_size'] = len(df_val_in)
    # print(idx, s_val_in)

    df_independent_metrics = df_independent_metrics.append(s_val_in, ignore_index=True)
    del s_val_in
df_independent_metrics.to_csv('test_independent_stations.csv')

<timed exec>:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
<timed exec>:27: FutureWarning: The frame.append method is deprecated and will be removed from panda

CPU times: user 8.48 s, sys: 223 ms, total: 8.7 s
Wall time: 13.6 s


<timed exec>:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
